In [1]:
#STIX simulator

In [1]:
import sys
sys.path.append('.')
import glob
import numpy as np
import math
from core.grid_parameters import nominal as ngd


import matplotlib.pyplot as plt


from ipywidgets import interact, interact_manual, widgets


from IPython.display import clear_output

from core.stix_imager import StixImager
from core import stix_grid as sgd
from core import stix_cfl as cfl
from core import stix_detector as sdt
print(sdt.StixDetector.pixel_areas)

%matplotlib notebook

[9.282 9.282 9.282 9.282 0.903 0.903 0.903 0.903 9.282 9.282 9.282 9.282]


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
detector = sdt.StixDetector
det_pixel_polygons = detector.get_pixel_polygons()       


In [4]:
def sim_cfl(sun_x,sun_y):
    cf=cfl.CFL()
    cf.project(sun_x,sun_y)
    cf.plot()


In [5]:
plt.close('all')
maxPhase= 0.457628#gd['26']['FrontPitch']
interact_manual(sim_cfl, 
         sun_x=widgets.IntSlider(min=-3600,max=3600,value=0, step=1),
         sun_y=widgets.IntSlider(min=-3600,max=3600,value=0,step=1)
      
        
         
        )

interactive(children=(IntSlider(value=0, description='sun_x', max=3600, min=-3600), IntSlider(value=0, descrip…

<function __main__.sim_cfl(sun_x, sun_y)>

In [ ]:
grid = sgd.StixGrid()
grid.set_parameter_type('real')
det=19
plt.figure()
ax=plt.gca()
grid.create(det, which_grid='rear', user_phase=0, user_rot_deg=0)

grid.project()
grid.plot(ax)
plt.show()
strips=grid.strip_polygons
strips.shape


In [ ]:
im = StixImager(21)
im.create()
im.plot()
plt.show()

In [ ]:
from ROOT import TFile, TTree
from array import array

In [ ]:



f = TFile('stix_grid_parameters.root','recreate')
t = TTree('grids','grids')

is_front = array('i',[0])
det_idx = array('i',[0])
is_nominal = array('i',[0])
i_strip = array('i',[0])
polygon_x=array('f', [0]*5)
polygon_y=array('f', [0]*5)
t.Branch('is_front',is_front,'is_front/I')
t.Branch('det_idx',det_idx,'det_idx/I')
t.Branch('i_strip',i_strip,'i_strip/I')
t.Branch('polygon_x', polygon_x, "polygon_x[5]/F")
t.Branch('polygon_y', polygon_y, "polygon_y[5]/F")
t.Branch('is_nominal_parameters',is_nominal,'is_nominal_parameters/I')
fig,axs=plt.subplots(8,8)
ifig=0
for which in ['front','rear']:
    for i in range(32):
        if i==8 or i==9:
            continue
        grid = sgd.StixGrid()
   
        
        
        row= ifig%8
        col=int(ifig/8)
        ifig+=1
        try:
            grid_type='real'
            grid.set_parameter_type(grid_type)
            grid.create(i, which_grid=which, user_phase=0, user_rot_deg=0)
            
            

        except Exception as e:
            #raise
            grid_type='nominal'
            grid.set_parameter_type('nominal')
            grid.create(i, which_grid=which, user_phase=0, user_rot_deg=0)
            print(e)
            #continue
        #plt.show()
        strips=grid.strip_polygons
        is_nominal[0]=1 if grid_type == 'nominal' else 0
        det_idx[0]=i
        is_front[0]=1 if which=='front' else 0
        
        for i, strip in enumerate(strips):
            i_strip[0]=i
            for ip, point in enumerate(strip):
                polygon_x[ip]=point[0]
                polygon_y[ip]=point[1]
                
            #print(strip)
            
            t.Fill()


t.Write()
f.Close()
        

In [ ]:
#then copy the file to g4STIX/grid_data